In [24]:
import pandas as pd
import xarray as xr
import glob
import os
import netCDF4
import scipy
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import scale 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
#from oceans.filters import lanc  #pip install ocean
import cartopy as cart
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs

In [25]:
def low_pass_weights(window, cutoff):
    """Calculate weights for a low pass Lanczos filter.
    Args:
    window: int
        The length of the filter window.
    cutoff: float
        The cutoff frequency in inverse time steps.
    """
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

In [26]:
#/nfs/public_ads/Oelsmann/marcello/gesla_v2/gesla_v2_northsea_balticsea.nc


region = 'NORTHSEA'

start_time = '01/01/2004'

end_time = '31/12/2004'

if region in {'BALTIC'} :
    max_lat = 66.0
    min_lat = 53.0
    max_lon = 31.0
    min_lon = 9.0
    
    #tg_dac = xr.open_dataset('/nfs/DGFI8/H/work_marcello/giussani_machinelearning_data/TG_DACcorrected/baltic_tg_gesla_dac.nc')
    tg_dac = xr.open_dataset('/nfs/public_ads/Oelsmann/marcello/gesla_v2/gesla_v2_northsea_balticsea.nc')

    lon = np.asarray(tg_dac.lon.values,dtype=float)
    lat = np.asarray(tg_dac.lat.values,dtype=float)    
    
#     # Exclude tide gauges in Kattegat and Skagerrag, because out of the training area

    indices =  ~( (lon < 13.0) & (lon > 9) &   \
                              (lat < 66.0) & (lat > 56.0)) & ((lon < max_lon) & (lon > min_lon) &   \
                              (lat < max_lat) & (lat > min_lat))   
    

    # Altimetry
    alti_full = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_baltic2004_01to12.csv',index_col=[0],parse_dates=[0])
    
    #model
    copernicus_full = pd.read_csv('/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_copernicus.csv',index_col=[7],parse_dates=[7])
    copernicus_full['sla_predicted'] = copernicus_full['sla_predicted'] 

    #cmems
    cmems_full = pd.read_csv('/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_cmems_allsat.csv',index_col=[5],parse_dates=[5])    
    

elif region in {'NORTHSEA'} :
    
    max_lat = 61.0
    min_lat = 50.0
    max_lon = 12.2
    min_lon = -4.0  
    
    tg_dac = xr.open_dataset('/nfs/public_ads/Oelsmann/marcello/gesla_v2/gesla_v2_northsea_balticsea.nc')
    
    lon = np.asarray(tg_dac.lon.values,dtype=float)
    lat = np.asarray(tg_dac.lat.values,dtype=float)
    
    indices =  ( (lon < max_lon) & (lon > min_lon) &   \
                              (lat < max_lat) & (lat > min_lat) )    

    # Altimetry
    alti_full = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_northsea2004_01to12.csv',index_col=[0],parse_dates=[0])
    alti_full_k1 = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_northsea2004_01to12_k1.csv',index_col=[0],parse_dates=[0])
    alti_full_k3 = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_northsea2004_01to12_k3.csv',index_col=[0],parse_dates=[0])
    alti_full_k4 = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_northsea2004_01to12_k4.csv',index_col=[0],parse_dates=[0])
    alti_full_k5 = pd.read_csv(r'/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_northsea2004_01to12_k5.csv',index_col=[0],parse_dates=[0])

    
    #model
    copernicus_full = pd.read_csv('/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_copernicus.csv',index_col=[7],parse_dates=[7])
    copernicus_full['sla_predicted'] = copernicus_full['sla_predicted'] 

    #cmems
    cmems_full = pd.read_csv('/DGFI8/H/work_marcello/machine_learning_altimetry/test_prediction_newpoints_surge_cmems_NORTHSEA_allsat.csv',index_col=[5],parse_dates=[5])



tg_dac = tg_dac.sel({'x':indices })

tg_dac


<xarray.Dataset>
Dimensions:  (time: 32677, x: 35)
Coordinates:
    lon      (x) float64 1.751 -1.875 -3.975 0.05703 ... 11.8 8.433 1.318 -2.075
    lat      (x) float64 52.47 50.71 51.57 50.78 ... 57.68 55.47 51.12 57.14
  * time     (time) datetime64[ns] 1993-01-01 1993-01-01T06:00:00 ... 2015-05-15
    index    (x) int64 ...
Dimensions without coordinates: x
Data variables:
    sla      (time, x) float64 ...
Attributes:
    info:     GESLAv2 dataset; DAC and Tide removed (with 40-h loess filter).

In [27]:
# Distance vs TG
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    
    
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 -lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

    return np.round(res, 2)

In [28]:
# list_names = ['landsortnorra-010-sweden-smhi','gedser-gedser-denmark-dmi',\
#               'visby-026-sweden-smhi']

#list_names = ['skanor-019-sweden-smhi','kungsholmsfort-008-sweden-smhi',\
#              'simrishamn-017-sweden-smhi']

list_names = ['skanor-019-sweden-smhi']

corr_ML=np.empty(np.size(tg_dac.x))*np.nan
corr_ML_k1=np.empty(np.size(tg_dac.x))*np.nan
corr_ML_k3=np.empty(np.size(tg_dac.x))*np.nan
corr_ML_k4=np.empty(np.size(tg_dac.x))*np.nan
corr_ML_k5=np.empty(np.size(tg_dac.x))*np.nan
corr_cmems=np.empty(np.size(tg_dac.x))*np.nan

rms_ML=np.empty(np.size(tg_dac.x))*np.nan
rms_ML_k1=np.empty(np.size(tg_dac.x))*np.nan
rms_ML_k3=np.empty(np.size(tg_dac.x))*np.nan
rms_ML_k4=np.empty(np.size(tg_dac.x))*np.nan
rms_ML_k5=np.empty(np.size(tg_dac.x))*np.nan
rms_cmems=np.empty(np.size(tg_dac.x))*np.nan

std_ML=np.empty(np.size(tg_dac.x))*np.nan
std_ML_k1=np.empty(np.size(tg_dac.x))*np.nan
std_ML_k3=np.empty(np.size(tg_dac.x))*np.nan
std_ML_k4=np.empty(np.size(tg_dac.x))*np.nan
std_ML_k5=np.empty(np.size(tg_dac.x))*np.nan
std_cmems=np.empty(np.size(tg_dac.x))*np.nan
std_tg=np.empty(np.size(tg_dac.x))*np.nan


row_index = np.array([0,0,0])
#col_index = np.array([0,1,2])
col_index = np.arange(0,np.size(tg_dac.x))

#fig, axs = plt.subplots(np.size(tg_dac.x), 1, sharex=True, sharey=False, figsize=(10,100))

#fig_b, axs_b = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(12,12))

counter = 0


#for tide_gauge_name in list_names :

for count_TG in tg_dac.x[0:] :

    #radius_of_distance_km = 100
    #tide_gauge_name='visby-026-sweden-smhi'
    
    start_lat = np.asarray(tg_dac.lat[count_TG].values,dtype=float)
    start_lon = np.asarray(tg_dac.lon[count_TG].values,dtype=float)

#     if region in {'BALTIC'} :
#         tide_gauge_name = tg_dac.name[count_TG]
#         tg_due = tg_dac.where(tg_dac.name==tide_gauge_name, drop=True)
#         tide_gauge_name = tg_dac.name[count_TG]
#     else:
    tide_gauge_name = tg_dac.index[count_TG]
    tg_due = tg_dac
    

    ## TIDE CORRECTION

    # dates = []

    # for j in range(-1,24*28*365+5):
    #      dates.append(np.array([datetime(1993, 1, 1, 0, 0, 0) + 
    # timedelta(hours=1+j)]))

    # dates = np.ravel(dates)

    # plt.figure(figsize=(15,5),dpi=320)

    # tide_correction_fes = pd.read_csv("/DGFI8/H/work_marcello/giussani_machinelearning_data/TG/landsortnorra-010-sweden-smhi_fes_ocean.txt",header=None)/100

    # plt.plot(dates,tide_correction_fes,)

    # plt.plot(tg_due.date_time,tg_due.sealevel-tide_correction_fes)
    # plt.ylabel('m')




    # initialise data of lists.
#     if region in {'BALTIC'} :
#         data = {'time':tg_due.time.values,
#                 'sla_dac':np.squeeze(tg_due.sla_dac.values)}
#     else:
    data = {'time':tg_due.time.values,
            'sla_dac':tg_due.sla[:,count_TG]}    
    #break

    # Create DataFrame
    tg = pd.DataFrame(data)
    
    # Remove mean from the tide gauge
    mean_tg_due= tg["sla_dac"].mean()
    tg.sla_dac = tg.sla_dac - tg["sla_dac"].mean()    
    median_tg_due= tg["sla_dac"].median()
    mad_tg_due= tg["sla_dac"].mad()    
    
    # Remove outliers from the tide gauge    
    tg.loc[tg["sla_dac"] > median_tg_due+1.4826*2*mad_tg_due] = np.nan
    tg.loc[tg["sla_dac"] < median_tg_due-1.4826*2*mad_tg_due] = np.nan
    
    tg=tg.set_index(tg.time)
    tg=tg.loc['2004-01-01':'2004-12-31']
    
    tg = tg.groupby(pd.Grouper(freq='24H')).mean()

    #means_tg = tg.groupby(pd.Grouper(freq='3H')).mean()
    
    # ---------------------------------------------------------------------------------
    
    def alti_processing(alti_full) :

        alti=alti_full.loc['2004-01-15':'2004-12-15']
        distances_km = haversine_distance(start_lat, start_lon, alti.lat, alti.lon)
        alti['distances_km']=distances_km
        alti.dropna(subset = ["sla_predicted"], inplace=True)
        alti = alti.loc[alti['distances_km'] == np.nanmin(alti['distances_km'])]
        alti = alti.groupby(['time']).mean()
        
        return alti

    alti = alti_processing(alti_full)
    alti_k1 = alti_processing(alti_full_k1)
    alti_k3 = alti_processing(alti_full_k3)
    alti_k4 = alti_processing(alti_full_k4)
    alti_k5 = alti_processing(alti_full_k5)


    

    #alti

    copernicus=copernicus_full.loc['2004-01-15':'2004-12-15']
    cmems=cmems_full.loc['2004-01-15':'2004-12-15']
    
    

    # Compute distances from tide gauges 

    distances_km_copernicus = haversine_distance(start_lat, start_lon, copernicus.lat, copernicus.lon) 
    distances_km_cmems = haversine_distance(start_lat, start_lon, cmems.lat, cmems.lon)


    copernicus['distances_km']=distances_km_copernicus
    cmems['distances_km']=distances_km_cmems

    copernicus.dropna(subset = ["sla_predicted"], inplace=True)
    cmems.dropna(subset = ["sla_predicted"], inplace=True)
    

    #alti = alti[distances_km<radius_of_distance_km]
    #copernicus = copernicus[distances_km_copernicus<radius_of_distance_km]
    #cmems = cmems[distances_km_cmems<radius_of_distance_km]
    
    
    copernicus = copernicus.loc[copernicus['distances_km'] == np.nanmin(copernicus['distances_km'])]
    cmems = cmems.loc[cmems['distances_km'] == np.nanmin(cmems['distances_km'])]
    #cmems = cmems.loc[np.abs(cmems['distances_km'] - np.nanmin(alti['distances_km']))<5]
    

    copernicus = copernicus.groupby(['time']).mean()
    cmems = cmems.groupby(['time']).mean()
    
    
    #Check that time series are not too short and that they have a similar number of data
    if np.shape(alti)[0] < 120 or np.shape(cmems)[0] < 120 or np.shape(tg.sla_dac)[0]<120 or np.shape(tg.sla_dac.values[np.isnan(tg.sla_dac.values)])[0] > 60  :
        if counter == 0 :
            fig.legend()        
        counter = counter + 1
        continue
        
    if np.shape(tg.sla_dac.values[np.isnan(tg.sla_dac.values)])[0] > 60  :
        if counter == 0 :
            fig.legend()        
        counter = counter + 1
        continue        
    
    
    freq = 1./10  # Hours
    window_size = 60
    


    
    # LOESS FILTER IN TIME
#     lowess_sm = sm.nonparametric.lowess
#     points_to_filter = 10 #100 points along-track correspond to about 60 km
#     frac_loess = points_to_filter/np.size(alti.sla_predicted)
#     alti.sla_predicted = lowess_sm(np.asarray(alti.sla_predicted),np.asarray(alti.times),frac=frac_loess,it=3, return_sorted = False,missing='drop')
#     cmems.sla_predicted = lowess_sm(np.asarray(cmems.sla_predicted),np.asarray(cmems.times),frac=frac_loess,it=3, return_sorted = False,missing='drop')
    

    # Oversample daily cmems to hourly values
    #cmems_oversample = cmems
    #cmems_oversample = cmems.resample('3H').pad()
    
    
#     means_alti = alti.groupby(pd.Grouper(freq='24H')).mean()
#     means_alti.sla_predicted = np.convolve(means_alti.sla_predicted, np.ones(7)/7, mode='same')
    
#     means_copernicus  = copernicus.groupby(pd.Grouper(freq='24H')).mean()
#     means_cmems  = cmems.groupby(pd.Grouper(freq='24H')).mean()
#     means_tg = tg.groupby(pd.Grouper(freq='24H')).mean()    
    



    # Plot of the daily products
#     means_alti.sla_predicted.plot(ax=axs[row_index[counter],col_index[counter]],label="ML")
#     means_copernicus.sla_predicted.plot(ax=axs[row_index[counter],col_index[counter]],label="model")
#     means_cmems.sla_predicted.plot(ax=axs[row_index[counter],col_index[counter]],label="cmems altimetry")
#     means_tg.sealevel.plot(ax=axs[row_index[counter],col_index[counter]],label="tide gauge")
#     if counter == 0 :
#         fig.legend()
    
    
    
    #Correlation Analysis

    #HIGH RES

    def compute_correlation(alti,copernicus,cmems,tg) :
    
        data = [alti["sla_predicted"], copernicus["sla_predicted"], cmems["sla_predicted"], tg["sla_dac"]]

        headers = ["ML", "model", "cmems", "TG"]

        df = pd.concat(data, axis=1, keys=headers)

        correlation_df = df.corr()

        correlation_value = correlation_df.values[0,3]
        corr_cmems = correlation_df.values[2,3]
        
        return correlation_value,corr_cmems
        
   
    corr_ML[count_TG],corr_cmems[count_TG]=compute_correlation(alti,copernicus,cmems,tg)
    corr_ML_k1[count_TG],corr_cmems[count_TG]=compute_correlation(alti_k1,copernicus,cmems,tg)
    corr_ML_k3[count_TG],corr_cmems[count_TG]=compute_correlation(alti_k3,copernicus,cmems,tg)
    corr_ML_k4[count_TG],corr_cmems[count_TG]=compute_correlation(alti_k4,copernicus,cmems,tg)
    corr_ML_k5[count_TG],corr_cmems[count_TG]=compute_correlation(alti_k5,copernicus,cmems,tg)
    
    
    
    rms_ML[count_TG]=np.sqrt(((alti["sla_predicted"] - tg["sla_dac"]) ** 2).mean())
    rms_ML_k1[count_TG]=np.sqrt(((alti_k1["sla_predicted"] - tg["sla_dac"]) ** 2).mean())
    rms_ML_k3[count_TG]=np.sqrt(((alti_k3["sla_predicted"] - tg["sla_dac"]) ** 2).mean())
    rms_ML_k4[count_TG]=np.sqrt(((alti_k4["sla_predicted"] - tg["sla_dac"]) ** 2).mean())
    rms_ML_k5[count_TG]=np.sqrt(((alti_k5["sla_predicted"] - tg["sla_dac"]) ** 2).mean())
    rms_cmems[count_TG]=np.sqrt(((cmems["sla_predicted"] - tg["sla_dac"]) ** 2).mean())  
    
#     stddiff_ML[count_TG]=np.std(alti["sla_predicted"]) - np.std(tg["sla_dac"]) 
#     stddiff_cmems[count_TG]=  np.std(cmems["sla_predicted"]) - np.std(tg["sla_dac"])    
    
    std_ML[count_TG]=scipy.stats.iqr(alti["sla_predicted"],nan_policy='omit')#np.rms(alti["sla_predicted"]) 
    std_cmems[count_TG]=  scipy.stats.iqr(cmems["sla_predicted"],nan_policy='omit')#np.rms(cmems["sla_predicted"])
    std_tg[count_TG]=  scipy.stats.iqr(tg["sla_dac"],nan_policy='omit')#np.rms(tg["sla_dac"])    
    
    counter = counter + 1
    
    #break
    

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

...

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/passaro/anaconda3/envs/machine_learning_altimetry_validation/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

In [29]:
print('Mean difference in correlation ML - CMEMS (%) k1')
print(np.nanmean((corr_ML_k1 - corr_cmems) / corr_cmems * 100))
print('Mean difference in correlation ML - CMEMS (%) k2')
print(np.nanmean((corr_ML - corr_cmems) / corr_cmems * 100))
print('Mean difference in correlation ML - CMEMS (%) k3')
print(np.nanmean((corr_ML_k3 - corr_cmems) / corr_cmems * 100))
print('Mean difference in correlation ML - CMEMS (%) k4')
print(np.nanmean((corr_ML_k4 - corr_cmems) / corr_cmems * 100))
print('Mean difference in correlation ML - CMEMS (%) k5')
print(np.nanmean((corr_ML_k5 - corr_cmems) / corr_cmems * 100))

# RMS
print('Mean difference in RMS ML - CMEMS (%) k1')
print(np.nanmean((rms_ML_k1 - rms_cmems) / rms_cmems *100))
print('Mean difference in RMS ML - CMEMS (%) k2')
print(np.nanmean((rms_ML - rms_cmems) / rms_cmems *100))
print('Mean difference in RMS ML - CMEMS (%) k3')
print(np.nanmean((rms_ML_k3 - rms_cmems) / rms_cmems *100))
print('Mean difference in RMS ML - CMEMS (%) k4')
print(np.nanmean((rms_ML_k4 - rms_cmems) / rms_cmems *100))
print('Mean difference in RMS ML - CMEMS (%) k5')
print(np.nanmean((rms_ML_k5 - rms_cmems) / rms_cmems *100))





Mean difference in correlation ML - CMEMS (%) k1
9.241931817447846
Mean difference in correlation ML - CMEMS (%) k2
9.983182601031883
Mean difference in correlation ML - CMEMS (%) k3
9.861962249290261
Mean difference in correlation ML - CMEMS (%) k4
9.080609978191815
Mean difference in correlation ML - CMEMS (%) k5
9.600592553678037
Mean difference in RMS ML - CMEMS (%) k1
-5.791279621648563
Mean difference in RMS ML - CMEMS (%) k2
-6.992121313224745
Mean difference in RMS ML - CMEMS (%) k3
-6.517323949546272
Mean difference in RMS ML - CMEMS (%) k4
-5.83658968767943
Mean difference in RMS ML - CMEMS (%) k5
-6.1050811256307975


#### 